In [1]:
import jupyter_manim
import pandas as pd
# Example CSV reading (replace with your actual file path)
pgn = pd.read_csv("/Users/benplace/chess_com_games_2024-11-08_moves.csv")

def pgn_converter(pgn):
    def convert_castling_move(move, color, game_id):
        if color == 'White':  
            if move == "O-O": 
                return [(game_id, (7, 4), (7, 6)), (game_id, (7, 7), (7, 5))]
            elif move == "O-O-O":  
                return [(game_id, (7, 4), (7, 2)),(game_id, (7, 0), (7, 3))]
        elif color == 'Black':  
            if move == "O-O":  
                return [(game_id, (0, 4), (0, 6)),(game_id, (0, 7), (0, 5))]
            elif move == "O-O-O":
                return [(game_id, (0, 4), (0, 2)),(game_id, (0, 0), (0, 3))]
        return None

    expanded_moves = []
    
    for idx, row in pgn.iterrows():
        move = row['notation']
        color = row['color'] 
        game_id = row['game_id']

        if move in ["O-O", "O-O-O"]:  
            castling_moves = convert_castling_move(move, color, game_id)
            for castling_move in castling_moves:
                expanded_moves.append(castling_move)
        else:
            from_square = (8 - int(row['from_square'][1]), ord(row['from_square'][0]) - ord('a'))
            to_square = (8 - int(row['to_square'][1]), ord(row['to_square'][0]) - ord('a'))
            expanded_moves.append((game_id, from_square, to_square))
    expanded_df = pd.DataFrame(expanded_moves, columns=['game_id', 'from_square', 'to_square'])
    return expanded_df

pgn_moves = pgn_converter(pgn)
game_moves = (pgn_moves.groupby('game_id').apply(lambda x: list(zip(x['from_square'], x['to_square'])), include_groups=False).to_dict())
game_moves_1 = [((0,0),(1,1)),((7,7),(6,6))]



In [5]:
%%manim -qh -v WARNING ChessWithText

from manim import *
from chess_board import ChessBoard
import numpy as np
import pandas as pd

class ChessWithText(Scene):
    def construct(self):

        # Animating the 5th November game
        board = ChessBoard("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
        board.scale(0.5)
        chess_group = Group(board).move_to(ORIGIN)
        self.play(FadeIn(chess_group))
        self.wait(1)
        self.play(chess_group.animate.shift(RIGHT * 3))

        text1 = Text("This is a \"bullet\" chess game\n I played on 5th November 2024").shift(LEFT * 3 + UP*2)
        text1.scale(0.65)
        self.play(Write(text1))
        text2 = Text("In this game I played 57 moves \nin around 2 minutes on my phone").shift(LEFT * 3)
        text2.scale(0.65)
        text3 = Text("It marked the 20th game I played\nthat day and it got me wondering\n...how far has my thumb scrolled\naccross all my bullet games?").shift(LEFT * 3+DOWN*2)
        text3.scale(0.65)
        self.wait(1)
        
        move_no  = 0
        for move in game_moves['0f066139-cd3e-465f-8427-97035757bd63']:
            move_no += 1
            piece_move = board.move_piece(move[0][0], move[0][1], move[1][0], move[1][1])

            if move_no == 20:
                self.play(Write(text2),run_time = 1)
            if move_no == 50: 
                self.play(Write(text3),run_time = 1)

            self.play(*piece_move, run_time=0.01)  
            self.wait(0.2) 

        self.clear()       

        # Animating how large the chess squares are
        text4 = Text("We first need to figure out how far my thumb scrolls for each move").shift(UP*3)
        text4.scale(0.7)
        self.play(Write(text4))

        text5 = Text("Say we have a phone with dimensions\n15.0 cm x 7.5cm and the chess board\nhas dimensions 6.4cm x 6.4cm.").shift(LEFT*3)
        text5.scale(0.65)

        phone = Rectangle(width=1, height=2)
        phone.set_stroke(WHITE, width=2) 
        phone.shift(RIGHT*3)
        phone.scale(2.5)

        phone_chess_board = Square()
        phone_chess_board.shift(RIGHT*3)

        self.play(Write(text5),Create(phone),Create(phone_chess_board),run_time = 4)
        self.wait(1)

        phone_chess_square = Square()
        phone_chess_square.set_fill(WHITE, opacity  = 0.7)
        phone_chess_square.scale(0.125)
        phone_chess_square.move_to(phone_chess_board.get_corner(DL) + UP * phone_chess_square.height / 2 
                                   + RIGHT * phone_chess_square.width / 2)

        text6 = Text("Then each square has length 0.8cm").shift(LEFT*3)
        text6.scale(0.65)

        self.play(FadeOut(text5))

        self.play(Write(text6),Create(phone_chess_square),run_time = 1)

        self.wait(2)

        board = ChessBoard("8/8/8/8/8/8/8/8")
        board.scale(0.25)

        chess_group = Group(board).move_to(RIGHT * 3)
        
        self.play(FadeOut(phone),FadeOut(phone_chess_square),FadeOut(phone_chess_board),FadeIn(chess_group),FadeOut(text6))

        
        # Animating different number of moves for each piece
        text7 = Text("We first need to consider that\neach move is different, for example,").shift(LEFT*3,UP*1)
        text7.scale(0.65)

        self.play(chess_group.animate.scale(2),Write(text7),run_time = 1)

        text8 = Text("a Pawn can move 1 or 2 squares,").shift(LEFT*3)
        text8.scale(0.65)
        board = ChessBoard("8/8/8/8/8/8/3PP3/8")
        arrows = [(6, 4, -2, 0), (6, 3, -1, 0)]
        for arrow in arrows:
            board.add_arrow(*arrow)
        chess_group_with_arrows = Group(board).move_to(RIGHT * 3)
        chess_group_with_arrows.scale(0.5)
        self.play(FadeIn(chess_group_with_arrows),Write(text8),run_time = 1)
        self.wait(0.2)
        self.play(FadeOut(text8),FadeOut(chess_group),run_time = 1)

        text9 = Text("a Rook can move 1 to 7 squares,").shift(LEFT*3)
        text9.scale(0.65)
        board = ChessBoard("8/8/8/8/8/2R5/8/8")
        arrows = [(5, 2, -5, 0), (5, 2, 2, 0),(5, 2, 0, -2),(5, 2, 0, 5)]
        for arrow in arrows:
            board.add_arrow(*arrow)
        chess_group_with_arrows1 = Group(board).move_to(RIGHT * 3)
        chess_group_with_arrows1.scale(0.5)
        self.play(FadeIn(chess_group_with_arrows1),Write(text9),FadeOut(chess_group_with_arrows),run_time = 1)
        self.wait(0.2)
        self.play(FadeOut(text9),run_time = 1)

        text10 = Text("a Knight can move in an \"L\" shape,").shift(LEFT*3)
        text10.scale(0.65)
        board = ChessBoard("8/8/8/8/3N4/8/8/8")
        arrows = [(4, 3, -2, 1), (4, 3, -2, -1),(4, 3, -1, -2),(4, 3, 1, -2),(4, 3, -1, 2),(4, 3, 1, 2),(4, 3, 2, -1),(4, 3, 2, 1)]
        for arrow in arrows:
            board.add_arrow(*arrow)
        chess_group_with_arrows2 = Group(board).move_to(RIGHT * 3)
        chess_group_with_arrows2.scale(0.5)
        self.play(FadeIn(chess_group_with_arrows2),Write(text10),FadeOut(chess_group_with_arrows1),run_time = 1)
        self.wait(0.2)
        self.play(FadeOut(text10),run_time = 1)

        text11 = Text("a King can move 1 square,").shift(LEFT*3)
        text11.scale(0.65)
        board = ChessBoard("8/8/5K2/8/8/8/8/8")
        arrows = [(2, 5, -1, 0), (2, 5, -1, 1),(2, 5, -1, -1),(2, 5, 1, 0),(2, 5, 1, 1),(2, 5, 1, -1),(2, 5, 0, 1),(2, 5, 0, -1)]
        for arrow in arrows:
            board.add_arrow(*arrow)
        chess_group_with_arrows3 = Group(board).move_to(RIGHT * 3)
        chess_group_with_arrows3.scale(0.5)
        self.play(FadeIn(chess_group_with_arrows3),Write(text11),FadeOut(chess_group_with_arrows2),run_time = 1)
        self.wait(0.2)
        self.play(FadeOut(text11),run_time = 1)

        text12 = Text("a Bishop can move diagonally,\n1 to 7 squares,").shift(LEFT*3)
        text12.scale(0.65)
        board = ChessBoard("8/1B6/8/8/8/8/8/8")
        arrows = [(1, 1, -1, -1), (1, 1, 6, 6),(1, 1,-1, 1),(1, 1, 1, -1)]
        for arrow in arrows:
            board.add_arrow(*arrow)
        chess_group_with_arrows4 = Group(board).move_to(RIGHT * 3)
        chess_group_with_arrows4.scale(0.5)
        self.play(FadeIn(chess_group_with_arrows4),Write(text12),FadeOut(chess_group_with_arrows3),run_time = 1)
        self.wait(0.2)
        self.play(FadeOut(text12),run_time = 1)

        text13 = Text("and a Queen can move in many ways").shift(LEFT*3)
        text13.scale(0.65)
        board = ChessBoard("8/8/8/4Q3/8/8/8/8")
        arrows = [(3, 4, 0, -4), (3, 4, 0, 3),(3, 4, -3, 0),(3, 4, 4, 0),(3, 4, 4, 3),(3, 4, 4, -4),(3, 4, -3, 3),(3, 4, -3, -4)]
        for arrow in arrows:
            board.add_arrow(*arrow)
        chess_group_with_arrows5 = Group(board).move_to(RIGHT * 3)
        chess_group_with_arrows5.scale(0.5)
        self.play(FadeIn(chess_group_with_arrows5),Write(text13),FadeOut(chess_group_with_arrows4),run_time = 1)
        self.wait(1)
        self.play(FadeOut(text13),FadeOut(text7),FadeOut(chess_group_with_arrows5),run_time = 1)

        board_w = ChessBoard("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
        board_w.scale(0.5)
        chess_group_w = Group(board_w).move_to(RIGHT * 3)
        self.play(FadeIn(chess_group_w))

        text14 = Text("We also need to get the distances\ndepending on whether we are").shift(LEFT*3+UP*1)
        text14.scale(0.65)
        text15 = Text("White").shift(LEFT*3)
        text15.scale(0.65)
        text16 = Text("or Black").shift(LEFT*3)
        text16.scale(0.65)

        self.play(Write(text14))
        self.play(Write(text15))
        self.wait(1)
        self.play(FadeOut(text15))
        board_b = ChessBoard("PPPPPPPP/RNBQKBNR/8/8/8/8/rnbqkbnr/pppppppp w KQkq - 0 1")
        board_b.scale(0.5)
        chess_group_b = Group(board_b).move_to(RIGHT * 3)
        self.play(Write(text16),FadeOut(chess_group_w),FadeIn(chess_group_b))
        self.wait(1)
        self.play(FadeOut(chess_group_b),FadeOut(text4),FadeOut(text16),FadeOut(text14))

        
        # Animating the distance scrolled for one game
        text17 = Text("Now we calculate the Distance Scrolled for our game before").shift(UP*3)
        text17.scale(0.65)

        board = ChessBoard("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
        board.scale(0.5)
        chess_group = Group(board).move_to(RIGHT * 3)

        distance = 0
        text_distance = Text("{:.2f} cm".format(distance)).shift(LEFT*3)
        self.play(FadeIn(chess_group),Write(text_distance),Write(text17))

        for move in game_moves['0f066139-cd3e-465f-8427-97035757bd63']:
            piece_move = board.move_piece(move[0][0], move[0][1], move[1][0], move[1][1])
            distance += 0.4*np.sqrt((move[1][0]-move[0][0])**2 + (move[0][1]-move[1][1])**2)
            self.play(FadeOut(text_distance), run_time = 0.005)
            text_distance = Text("{:.2f} cm".format(distance)).shift(LEFT*3)
            self.play(*piece_move, run_time=0.02)
            self.play(Write(text_distance),run_time=0.005)

        
        self.wait(1)
        self.clear()


        # Animating the average scrolling distance and total scrolling distance
        text_distance.shift(RIGHT*3)
        text20 = Text("In the game we scrolled roughly").shift(UP*2)
        text_distance.scale(1)

        self.play(FadeIn(text_distance),FadeIn(text20),run_time = 1)
        self.wait(1)
        self.play(text_distance.animate.shift(RIGHT*1+UP*2),FadeOut(text20))
        self.play(text_distance.animate.scale(0.65))

        text21 = Text("We then find the distances\nmoved accross 8 more games,").shift(LEFT*4)
        text21.scale(0.7)
        text22 = Text("72.44 cm").shift(RIGHT*1)
        text22.scale(0.65)
        text23 = Text("88.79 cm").shift(RIGHT*1+DOWN*2)
        text23.scale(0.65)
        text24 = Text("75.45 cm").shift(RIGHT*3+UP*2)
        text24.scale(0.65)
        text25 = Text("80.56 cm").shift(RIGHT*3)
        text25.scale(0.65)
        text26 = Text("85.68 cm").shift(RIGHT*3+DOWN*2)
        text26.scale(0.65)
        text27 = Text("63.68 cm").shift(RIGHT*5+UP*2)
        text27.scale(0.65)
        text28 = Text("66.37 cm").shift(RIGHT*5)
        text28.scale(0.65)
        text29 = Text("99.28 cm").shift(RIGHT*5+DOWN*2)
        text29.scale(0.65)
        
        self.play(Write(text21))
        self.play(Write(text22),Write(text23),Write(text24),Write(text25),Write(text26),Write(text27),Write(text28),Write(text29),run_time = 2)
        self.play(FadeOut(text21))
        text30 = Text("take an average to an estimate\nfor distance scrolled per game,").shift(LEFT*4)
        text30.scale(0.7)
        self.play(Write(text30))
        text31 = Text("81.17 cm").shift(RIGHT*3)
        self.play(text_distance.animate.move_to(RIGHT*3).fade(1),text22.animate.move_to(RIGHT*3).fade(1),text23.animate.move_to(RIGHT*3).fade(1),
                  text24.animate.move_to(RIGHT*3).fade(1),text25.animate.move_to(RIGHT*3).fade(1),text26.animate.move_to(RIGHT*3).fade(1),
                  text27.animate.move_to(RIGHT*3).fade(1),text28.animate.move_to(RIGHT*3).fade(1),text29.animate.move_to(RIGHT*3).fade(1),
                  FadeIn(text31), run_time = 1.5)

        self.play(FadeOut(text30),run_time = 0.5)
        text40 = Text("and then multiplying by\n16,461 (the total number\nof games played)").shift(LEFT*3)
        text40.scale(0.7)
        text41 = Text("x 16461").shift(RIGHT*3+DOWN*0.5)
        self.play(Write(text40),text31.animate.shift(UP*0.5),Write(text41))
        self.wait(1)
        text42 = Text("13,363 m").shift(UP*1)                
        text43 = Text("This gives us a distance scrolled of").shift(UP*2)
        text43.scale(0.7)

        text44 = Text("which is equivalent to 127 football fields!").shift(DOWN*1)
        text44.scale(0.9)
        self.play(text41.animate.move_to(UP*1).fade(1),text31.animate.move_to(UP*1).fade(1),
                  FadeIn(text43),FadeOut(text40),FadeIn(text42))
        self.wait(1)
        self.play(FadeIn(text44))
        self.wait(2)



Manim Community v0.18.1